In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim

     |████████████████████████████████| 962 kB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 61.7 MB/s eta 0:00:01


In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
read_table = pd.read_html(url, header=[0])
df = read_table[0]
df.head()

,Name,Status,PopulationCensus2000-05-15,PopulationCensus2011-01-09,Unnamed: 4
0,Arima,Borough,32278,33606,→
1,Arima Proper,Urban Community,9129,9082,→
2,Calvary Hill,Urban Community,1165,1330,→
3,Carib Homes,Urban Community,465,585,→
4,Malabar,Urban Community,8218,11130,→


In [12]:
df.drop(['PopulationCensus2000-05-15', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={'PopulationCensus2011-01-09':'Population'}, inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218
608,Warren Village,Urban Community,2125


In [13]:
df.drop([609], inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
604,Tunapuna,Urban Community,16701
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218


In [34]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent='myGeocoder')
location = []
#geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#for i in range(5):
address = df.Name[i]
    
   # coords = None
    
    #while(coords is None):
g=locator.geocode('{}, Trinidad and Tobago'.format(address))
        
location.append(g)
print(location)

[Location(Arima, 300831, Trinidad and Tobago, (10.6372057, -61.2829762, 0.0))]


In [14]:
#for i in range(df.shape[0]):
df['Address'] = df['Name'].astype(str) + ', Trinidad and Tobago'
df.head()

,Name,Status,Population,Address
0,Arima,Borough,33606,"Arima, Trinidad and Tobago"
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago"
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago"
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago"
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago"


In [16]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent='myGeocoder')
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('White Land, Trinidad and Tobago',), **{}).
Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 1369, in getresponse
    response.begin()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/socket.py", line 589, in rea

In [17]:
print(df.shape)
df.head()

(609, 9)


,Name,Status,Population,Address,location,point,latitude,longitude,altitude
0,Arima,Borough,33606,"Arima, Trinidad and Tobago","(Arima, 300831, Trinidad and Tobago, (10.63720...","(10.6372057, -61.2829762, 0.0)",10.637206,-61.282976,0.0
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago",None,None,NaN,NaN,NaN
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago","(Calvary Hill, Saint Ann's, Port of Spain, 190...","(10.7097744, -61.5090987, 0.0)",10.709774,-61.509099,0.0
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago",None,None,NaN,NaN,NaN
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago","(Malabar, Arima, 300831, Trinidad and Tobago, ...","(10.6196194, -61.2774636, 0.0)",10.619619,-61.277464,0.0


In [35]:
df.dropna(inplace=True)
df.drop(['Address', 'location', 'point', 'altitude'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Name,Status,Population,latitude,longitude
0,Arima,Borough,33606,10.637206,-61.282976
1,Calvary Hill,Urban Community,1330,10.709774,-61.509099
2,Malabar,Urban Community,11130,10.619619,-61.277464
3,Mount Pleasant,Urban Community,1240,11.167020,-60.792943
4,O'Meara Road,Urban Community,2593,10.592523,-61.294142


In [36]:
df.shape

(479, 5)

In [22]:
import json # library to handle JSON files
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium
import folium

!pip install geopy
from geopy.geocoders import Nominatim

from folium import plugins

print('Libraries imported.')

Libraries imported.


In [40]:
tt_address = 'Trinidad and Tobago'

location = locator.geocode(tt_address)
tt_latitude = location.latitude
tt_longitude = location.longitude
print(str(tt_latitude) + ',' + str(tt_longitude))

10.8677845,-60.9821067


In [41]:
tt_map = folium.Map(location=[tt_latitude, tt_longitude], zoom_start=12)

for lat, lng, status, population, name in zip(df['latitude'], df['longitude'], df['Status'], df['Population'], df['Name']):
    label = '{}: {}, {}'.format(name, status, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    parse_html=False).add_to(tt_map)
    
tt_map

In [43]:
# The code was removed by Watson Studio for sharing.

In [44]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
tt_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Arima
Calvary Hill
Malabar
Mount Pleasant
O'Meara Road
Tumpuna Road
Chaguanas
Charlieville
Edinburgh 500
Edinburgh Gardens
Endeavour Village
Enterprise
Esmeralda
Felicity
Homeland Gardens
Jerningham Junction
Lange Park
Lendore Village
Montrose Village
Munroe Settlement
Petersfield
Couva/Tabaquite/Talparo
Agostini Village
Arena
Balmain
Basta Hall
Bonne Aventure
Brasso Caparo Village
Brasso Tamana
Brasso Venado
Brechin Castle
Brickfield
Brothers Road
Butler Village
Calcutta Road No. 2
Calcutta Settlement No. 2
California
Caparo
Carapichaima
Caratal
Carlsen Field
Cedar Hill
Chandernagore
Chase Village
Chickland
Chin Chin
Claxton Bay
Coalmine
Corosal
Couva Central
Diamond
Dow Village
Edinburgh Village
Esperanza
Fairview
Farnum Village
Felicity Hall
Flanagin Town
Forres Park
Freeport
Friendship
Gasparillo
Gran Couva
Guaracara
Hermitage
Indian Trail
Macaulay
Madras Settlement
Mamoral No. 2
Mayo
Mc Bean
Mount Pleasant
Mundo Nuevo
Nancoo Village
Orange Valley
Palmiste
Parforce
Pepper Village
P

In [46]:
print(tt_venues.shape)
tt_venues.head()

(1560, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arima,10.637206,-61.282976,Pennywise cosmetics,10.639299,-61.280745,Cosmetics Shop
1,Arima,10.637206,-61.282976,Pizza Hut,10.639227,-61.281888,Pizza Place
2,Arima,10.637206,-61.282976,Bo's Chinese Restaurant,10.635889,-61.281402,Chinese Restaurant
3,Arima,10.637206,-61.282976,Pizza Hut - Windsor,10.639398,-61.282298,Pizza Place
4,Arima,10.637206,-61.282976,Hi Lo Ridgewood,10.638998,-61.282862,Supermarket


In [48]:
tt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alyce Glen,6,6,6,6,6,6
Aranguez,5,5,5,5,5,5
Arena,1,1,1,1,1,1
Arima,9,9,9,9,9,9
Aripero Village,1,1,1,1,1,1
...,...,...,...,...,...,...
Waddle Village,4,4,4,4,4,4
Wellington,3,3,3,3,3,3
West Moorings,2,2,2,2,2,2


In [50]:
print('There are {} uniques categories.'.format(len(tt_venues['Venue Category'].unique())))

There are 181 uniques categories.


In [51]:
# one hot encoding
tt_onehot = pd.get_dummies(tt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tt_onehot['Neighborhood'] = tt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tt_onehot.columns[-1]] + list(tt_onehot.columns[:-1])
tt_onehot = tt_onehot[fixed_columns]

tt_onehot.head()

,Yoga Studio,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Travel & Transport,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Water Park,Waterfall,Waterfront,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
tt_onehot.shape

(1560, 181)

In [54]:
tt_grouped = tt_onehot.groupby('Neighborhood').mean().reset_index()
tt_grouped

,Neighborhood,Yoga Studio,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Travel & Transport,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Water Park,Waterfall,Waterfront,Wine Shop,Wings Joint,Women's Store
0,Alyce Glen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Aranguez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.2,0.0
2,Arena,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Arima,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Aripero Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,Waddle Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
295,Wellington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0
296,West Moorings,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
297,Whim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [55]:
tt_grouped.shape

(299, 181)

In [56]:
num_top_venues = 5

for hood in tt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tt_grouped[tt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alyce Glen----
                 venue  freq
0    Indian Restaurant  0.17
1   Chinese Restaurant  0.17
2  Fried Chicken Joint  0.17
3          Supermarket  0.17
4             Pharmacy  0.17


----Aranguez----
                           venue  freq
0             Chinese Restaurant   0.4
1              Indian Restaurant   0.2
2                    Wings Joint   0.2
3                 Breakfast Spot   0.2
4  Paper / Office Supplies Store   0.0


----Arena----
                           venue  freq
0                  Grocery Store   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                      Multiplex   0.0
4                    Music Store   0.0


----Arima----
                venue  freq
0         Pizza Place  0.22
1                Café  0.11
2      Cosmetics Shop  0.11
3  Chinese Restaurant  0.11
4         Supermarket  0.11


----Aripero Village----
           venue  freq
0     Sports Bar   1.0
1  Movie Theater   0.0
2  Moving Target   0.0
3 

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tt_venues_sorted = pd.DataFrame(columns=columns)
tt_venues_sorted['Neighborhood'] = tt_grouped['Neighborhood']

for ind in np.arange(tt_grouped.shape[0]):
    tt_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tt_grouped.iloc[ind, :], num_top_venues)

tt_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alyce Glen,Sports Bar,Chinese Restaurant,Pharmacy,Indian Restaurant,Supermarket,Fried Chicken Joint,Eastern European Restaurant,Field,Fast Food Restaurant,Farmers Market
1,Aranguez,Chinese Restaurant,Wings Joint,Indian Restaurant,Breakfast Spot,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
2,Arena,Grocery Store,Women's Store,Diner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
3,Arima,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
4,Aripero Village,Sports Bar,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service,Electronics Store


In [62]:
# set number of clusters
kclusters = 5

tt_grouped_clustering = tt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0], dtype=int32)

In [66]:
# add clustering labels
#tt_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tt_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tt_merged = tt_merged.join(tt_venues_sorted.set_index('Neighborhood'), on='Name')

tt_merged.head() # check the last columns!

,Name,Status,Population,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arima,Borough,33606,10.637206,-61.282976,1.0,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
1,Calvary Hill,Urban Community,1330,10.709774,-61.509099,1.0,Fast Food Restaurant,Gas Station,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Service
2,Malabar,Urban Community,11130,10.619619,-61.277464,0.0,Bar,Deli / Bodega,BBQ Joint,Women's Store,Event Service,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
3,Mount Pleasant,Urban Community,1240,11.167020,-60.792943,1.0,Auto Garage,Stadium,Diner,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,O'Meara Road,Urban Community,2593,10.592523,-61.294142,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [68]:
tt_merged.dropna(inplace=True)
# create map
map_clusters = folium.Map(location=[tt_latitude, tt_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['latitude'], tt_merged['longitude'], tt_merged['Name'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
tt_merged.loc[tt_merged['Cluster Labels'] == 0, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Urban Community,0.0,Bar,Deli / Bodega,BBQ Joint,Women's Store,Event Service,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Borough,0.0,Bar,Furniture / Home Store,Dessert Shop,Asian Restaurant,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
7,Urban Community,0.0,Café,Business Service,Cosmetics Shop,Bar,BBQ Joint,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
11,Urban Community,0.0,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
12,Urban Community,0.0,Coffee Shop,Bar,Lounge,Liquor Store,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
13,Urban Community,0.0,Bar,Nightclub,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
14,Urban Community,0.0,Coffee Shop,Bar,BBQ Joint,Liquor Store,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
16,Urban Community,0.0,Indian Restaurant,Gym / Fitness Center,Caribbean Restaurant,Bar,Park,Women's Store,Eastern European Restaurant,Field,Fast Food Restaurant,Farmers Market
17,Urban Community,0.0,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
32,Semi-Urban Community,0.0,Electronics Store,Caribbean Restaurant,Bar,Spa,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm


In [70]:
tt_merged.loc[tt_merged['Cluster Labels'] == 1, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Borough,1.0,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
1,Urban Community,1.0,Fast Food Restaurant,Gas Station,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Service
3,Urban Community,1.0,Auto Garage,Stadium,Diner,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
8,Urban Community,1.0,Convenience Store,Arcade,Burger Joint,Gym / Fitness Center,Bar,Furniture / Home Store,Event Service,Fish & Chips Shop,Field,Fast Food Restaurant
9,Urban Community,1.0,Chinese Restaurant,Fast Food Restaurant,Bakery,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
469,Urban Community,1.0,Spa,Event Service,Convenience Store,Middle Eastern Restaurant,Bar,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
471,Urban Community,1.0,Pharmacy,Chinese Restaurant,Bus Station,Fast Food Restaurant,Park,Basketball Court,Burger Joint,Bakery,Electronics Store,Women's Store
472,Urban Community,1.0,Pizza Place,Mobile Phone Shop,Pet Store,Electronics Store,Korean Restaurant,Playground,Cosmetics Shop,Convenience Store,Fast Food Restaurant,Farmers Market
474,Urban Community,1.0,Fast Food Restaurant,Cosmetics Shop,Indian Restaurant,Farmers Market,Park,Sandwich Place,Burger Joint,Electronics Store,Women's Store,Eastern European Restaurant


In [71]:
tt_merged.loc[tt_merged['Cluster Labels'] == 2, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
223,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
225,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
461,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service


In [72]:
tt_merged.loc[tt_merged['Cluster Labels'] == 3, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
38,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
53,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
59,Urban Community,3.0,Bar,Restaurant,Women's Store,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
68,Rural Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
75,Semi-Urban Community,3.0,Bar,Bakery,Women's Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
76,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
91,Semi-Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
92,Semi-Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
136,Region,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [73]:
tt_merged.loc[tt_merged['Cluster Labels'] == 4, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Urban Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
269,Rural Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
295,Rural Community,4.0,Beach,Scenic Lookout,Women's Store,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
318,Semi-Urban Community,4.0,Pool,Beach,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
334,Rural Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
358,Urban Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [34]:
from IPython.display import HTML
import base64 

def create_download_link( df, title = "Download CSV file", filename = "trinidad.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)

In [7]:
# The code was removed by Watson Studio for sharing.

,Name,Population,Serious Crimes Reported
0,Arima,33606,685
1,Chaguanas,83516,945
2,Couva/Tabaquite/Talparo,178410,1116
3,Diego Martin,102957,1023
4,Mayaro/Rio Claro,35650,199


In [30]:
!wget --quiet https://github.com/ssankar-23/coursera_capstone/blob/master/trinidad.json -O trinidad.json
print('Loaded.')

Loaded.


In [16]:
df_data_2.dtypes

Name                       object
Population                  int64
Serious Crimes Reported     int64
dtype: object

In [31]:
trinidad_geo = r'map.geojson' # geojson file

# create a plain world map
t_map = folium.Map(location=[10.536421, -61.311951], zoom_start=2, tiles='Mapbox Bright')

 

# display map
t_map

In [33]:
t_map.choropleth(
geo_data=trinidad_geo,
data=df_data_2,
columns=['Name', 'Population'],
key_on='feature.properties.Name',
fill_color='YlOrRd',
fill_opacity=0.7,
line_opacity=0.2,
legend_name='Trinidad Population by Municipality'
)

t_map

JSONDecodeError: Expecting value: line 7 column 1 (char 6)